# BEST BOOKS CHALLENGE

In [1]:
from selenium import webdriver
import pandas as pd
import re

In [2]:
def check_Series(title):
    if len(title.split('(')) > 1:
        return True
    return False

def getRatings(ratings):

    ratings = ratings.replace(',','')
    ratings = re.findall(r'\d+', ratings)
    avg_rating = float(ratings[0]+'.'+ratings[1])
    num_ratings = int(ratings[2])
    
    return avg_rating, num_ratings

def get_original_publish_year(original_publish_year):
    return  [int(s) for s in original_publish_year.replace(')','').split() if s.isdigit()][-1]
    
def get_no_review_pages(review_pages):
    return int(review_pages.replace(',','').split()[0])

In [3]:
my_driver = "chromedriver.exe"
op = webdriver.ChromeOptions()
op.add_argument('headless')

main_url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever'

#collect in dictionary
content_dict = {'url':[],'title':[],'author':[],'num_reviews':[],'num_ratings':[],'avg_rating':[],'num_pages':[],\
    'original_publish_year':[],'series':[],'genres':[],'awards':[],'places':[]}

with open('best-books.csv','w') as file:
    file.write(','.join(content_dict.keys()))
    file.write('\n')

k=0
page_number=0
save_freq = 10
no_items_needed = 5000

#move through the pages
while True:
    print("="*20+"page("+str(page_number)+")"+"="*20)
    main_driver = webdriver.Chrome(my_driver,options=op)
    main_driver.implicitly_wait(1)
    main_driver.get(main_url)

    books = main_driver.find_elements_by_xpath('.//a[@class="bookTitle"]')
    author = main_driver.find_elements_by_xpath('.//a[@class="authorName"]')
    ratings = main_driver.find_elements_by_xpath('.//span[@class="minirating"]')

    for it in range(len(books)):#len(books)
        #back to base url
        main_driver = webdriver.Chrome(my_driver,options=op)
        main_driver.implicitly_wait(1)
        main_driver.get(main_url)

        content_dict['title'].append(books[it].text)
        content_dict['url'].append(books[it].get_attribute("href"))
        content_dict['series'].append(check_Series(books[it].text))
        content_dict['author'].append(author[it].text)

        avg_rating, num_ratings = getRatings(ratings[it].text)
        content_dict['avg_rating'].append(avg_rating)
        content_dict['num_ratings'].append(num_ratings)

        #Follow the link of book i (redirection to book i page)
        #update movie_i_driver url
        book_i_driver_url = books[it].get_attribute("href")
        main_driver = webdriver.Chrome(my_driver,options=op)
        main_driver.implicitly_wait(1)
        main_driver.get(book_i_driver_url)

        bookMeta = main_driver.find_element_by_xpath('.//div[@id="bookMeta"]')
        hyperlink = bookMeta.find_elements_by_xpath('.//a[@class="gr-hyperlink"]')
        numberOfPages = main_driver.find_element_by_xpath('.//span[@itemprop="numberOfPages"]')

        details = main_driver.find_element_by_xpath('.//div[@id="details"]')
        original_publish_year = details.find_elements_by_xpath('.//div[@class="row"]')[1]
        main_driver.find_element_by_xpath('.//a[@id="bookDataBoxShow"]').click()

        bookDataBox = main_driver.find_element_by_xpath('.//div[@id="bookDataBox"]')

        try:
            setting_div = bookDataBox.find_element_by_xpath("//div[text()='Setting']")
            settings = setting_div.find_element_by_xpath("./following-sibling::div")
            places = settings.find_elements_by_tag_name("a")
            places = [ p.text for p in places if p.text != '…more' and p.text.strip() != '']
        except:
            places = None

        try:
            literary_awards_div = bookDataBox.find_element_by_xpath("//div[text()='Literary Awards']")
            literary_awards = literary_awards_div.find_element_by_xpath("./following-sibling::div")
            awards = literary_awards.find_elements_by_tag_name("a")
            awards = [ a.text for a in awards if a.text != '...more' and a.text.strip() != '']
        except:
            awards = None

        # #genres_div = main_driver.find_element_by_xpath("//div[text()='Genres']")
        # right_container = main_driver.find_element_by_xpath('.//div[@class="rightContainer"]')
        # bigBoxBody = right_container.find_element_by_xpath('//div[@class="bigBoxContent containerWithHeaderContent"]')
        # #class="h2Container gradientHeaderContainer"
        # print('the')
        # #print(genres_div.text)
        # #genres = genres_div.find_element_by_xpath("./following-sibling::div")
        # genres = bigBoxBody.find_elements_by_xpath('.//a[@class="actionLinkLite bookPageGenreLink"]')
        # print(genres)
        # #genres = [g.find_elements_by_xpath('.//div[@class="left"]').text for g in genres]

        content_dict['num_reviews'].append( get_no_review_pages(hyperlink[-1].text) )
        content_dict['num_pages'].append( get_no_review_pages(numberOfPages.text) )
        content_dict['original_publish_year'].append( get_original_publish_year(original_publish_year.text) )
        content_dict['places'].append(places)
        content_dict['awards'].append(awards)
        content_dict['genres'].append(None)

        #save data to csv after save_freq iterations
        if k%save_freq == 0 and k != 0:
            
            df = pd.DataFrame(content_dict)
            df.to_csv('best-books.csv', mode='a', header=False, index=False)
            print("writen "+str(k)+" items")
            content_dict = {'url':[],'title':[],'author':[],'num_reviews':[],'num_ratings':[],'avg_rating':[],'num_pages':[],\
                'original_publish_year':[],'series':[],'genres':[],'awards':[],'places':[]}
        k += 1
        
        
        #move to next page
    try:
        #back to base url
        main_driver = webdriver.Chrome(my_driver,options=op)
        main_driver.implicitly_wait(1)
        main_driver.get(main_url)
        
        #next_page = main_driver.find_element_by_xpath('//body[@id="styleguide-v2"]')
        
        next_page = main_driver.find_element_by_xpath('.//a[@class="next_page"]')

        #update main_url for next movie page
        main_url = next_page.get_attribute("href")

        #Stop after no_items_needed
        if k >= no_items_needed:
            break
        
        
    except:
        #stop loop when there is no next page
        break

    page_number += 1

    
#write remaining items to csv
if k%save_freq != 0:
    df = pd.DataFrame(content_dict)
    df.to_csv('best-books.csv', mode='a', header=False, index=False)
    print("writen "+str(k)+" items")



print("Done")


====================page(0)====================
writen 10 items


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@id="bookMeta"]"}
  (Session info: headless chrome=94.0.4606.61)


## Load the csv file below

In [5]:
df = pd.read_csv('best-books.csv') 
print(df)
print("\nData Shape: ",df.shape)

                                                  url  \
0   https://www.goodreads.com/book/show/2767052-th...   
1   https://www.goodreads.com/book/show/2.Harry_Po...   
2   https://www.goodreads.com/book/show/2657.To_Ki...   
3   https://www.goodreads.com/book/show/1885.Pride...   
4   https://www.goodreads.com/book/show/41865.Twil...   
5   https://www.goodreads.com/book/show/19063.The_...   
6   https://www.goodreads.com/book/show/170448.Ani...   
7   https://www.goodreads.com/book/show/11127.The_...   
8   https://www.goodreads.com/book/show/30.J_R_R_T...   
9   https://www.goodreads.com/book/show/11870085-t...   
10  https://www.goodreads.com/book/show/18405.Gone...   

                                                title             author  \
0             The Hunger Games (The Hunger Games, #1)    Suzanne Collins   
1   Harry Potter and the Order of the Phoenix (Har...       J.K. Rowling   
2                               To Kill a Mockingbird         Harper Lee   
3          